Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [18]:
batch_size = 128 # each epoch we feed 128 samples (training images) into our model
hidden_layer_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_para = tf.placeholder(tf.float32)
  
  # Variables. For the first layer
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  # Parameters for the hidden layer, it's a fully connected layer
  weights_2 = tf.Variable(
      tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  # Use tf.clip_by_value to prevent taking log of almost 0 during computing cross entropy
  logits = tf.matmul(layer1_output, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    l2_para * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer_1, weights_2) + biases_2)
  test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer_1, weights_2) + biases_2)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, l2_para : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 675.135010
Minibatch accuracy: 5.5%
Validation accuracy: 27.9%
Minibatch loss at step 500: 198.156845
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 113.194092
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 69.062576
Minibatch accuracy: 77.3%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 41.481300
Minibatch accuracy: 82.0%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 25.061741
Minibatch accuracy: 92.2%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.600804
Minibatch accuracy: 83.6%
Validation accuracy: 86.4%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [22]:
batch_size = 128 # each epoch we feed 128 samples (training images) into our model
hidden_layer_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. For the first layer
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  # Parameters for the hidden layer, it's a fully connected layer
  weights_2 = tf.Variable(
      tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  # Use tf.clip_by_value to prevent taking log of almost 0 during computing cross entropy
  logits = tf.matmul(layer1_output, weights_2) + biases_2 
    # value in this logits is interpreted as unnormalized log probability, thus having no specific range
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer_1, weights_2) + biases_2)
  test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer_1, weights_2) + biases_2)

In [23]:
num_steps = 200

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = 0 # so that the model can only see the first batch of data
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 282.385498
Minibatch accuracy: 23.4%
Validation accuracy: 25.8%
Minibatch loss at step 10: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 20: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 30: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 40: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 50: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 60: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 70: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 80: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 90: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [24]:
batch_size = 128 # each epoch we feed 128 samples (training images) into our model
hidden_layer_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. For the first layer
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_layer_nodes]))
  # Parameters for the hidden layer, it's a fully connected layer
  weights_2 = tf.Variable(
      tf.truncated_normal([hidden_layer_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  layer1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  # Use tf.clip_by_value to prevent taking log of almost 0 during computing cross entropy
  dropout1 = tf.nn.dropout(layer1_output, 0.5)
  logits = tf.matmul(dropout1, weights_2) + biases_2 
    # value in this logits is interpreted as unnormalized log probability, thus having no specific range
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer_1, weights_2) + biases_2)
  test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer_1, weights_2) + biases_2)

In [25]:
num_steps = 200

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = 0 # so that the model can only see the first batch of data
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 549.741760
Minibatch accuracy: 7.8%
Validation accuracy: 23.3%
Minibatch loss at step 10: 3.120503
Minibatch accuracy: 96.9%
Validation accuracy: 66.9%
Minibatch loss at step 20: 0.669717
Minibatch accuracy: 98.4%
Validation accuracy: 68.9%
Minibatch loss at step 30: 0.257096
Minibatch accuracy: 99.2%
Validation accuracy: 67.6%
Minibatch loss at step 40: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 50: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.1%
Minibatch loss at step 60: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.1%
Minibatch loss at step 70: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.2%
Minibatch loss at step 80: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.8%
Minibatch loss at step 90: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Minibatch loss at step 100: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
M

After inserting a dropout layer, it becomes more difficult for the training acc to achieve 0 (overfitting). And this finally results in a about 5% performance enhancement on the test set.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [61]:
batch_size = 128 # each epoch we feed 128 samples (training images) into our model
hidden_layer_nodes1 = 1024
hidden_layer_nodes2 = 256

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_para = tf.placeholder(tf.float32)
    
  global_step = tf.Variable(0) # count the number of steps taken.
  
  # Variables. For the first layer
  weights_1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, num_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    )
  biases_1 = tf.Variable(tf.zeros([hidden_layer_nodes1]))
  # Parameters for the hidden layer, it's a fully connected layer
  weights_2 = tf.Variable(
      tf.truncated_normal([hidden_layer_nodes1, hidden_layer_nodes2], stddev=np.sqrt(2.0 / hidden_layer_nodes1)))
  biases_2 = tf.Variable(tf.zeros([hidden_layer_nodes2]))
  weights_3 = tf.Variable(
      tf.truncated_normal([hidden_layer_nodes2, num_labels], stddev=np.sqrt(2.0 / hidden_layer_nodes1)))
  biases_3 = tf.Variable(tf.zeros([num_labels]))  
    
  # Training computation.
  layer1_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  layer2_output = tf.nn.relu(tf.matmul(layer1_output, weights_2) + biases_2)
  dropout1 = tf.nn.dropout(layer2_output, 0.5)
  logits = tf.matmul(dropout1, weights_3) + biases_3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
   l2_para * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3)))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_layer_2 = tf.nn.relu(tf.matmul(valid_layer_1, weights_2) + biases_2)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer_2, weights_3) + biases_3)
  test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_layer_2 = tf.nn.relu(tf.matmul(test_layer_1, weights_2) + biases_2)  
  test_prediction = tf.nn.softmax(tf.matmul(test_layer_2, weights_3) + biases_3)

In [62]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, l2_para : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.336002
Minibatch accuracy: 10.2%
Validation accuracy: 29.5%
Minibatch loss at step 500: 1.097791
Minibatch accuracy: 85.2%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.845775
Minibatch accuracy: 89.1%
Validation accuracy: 86.7%
Minibatch loss at step 1500: 0.727886
Minibatch accuracy: 86.7%
Validation accuracy: 87.2%
Minibatch loss at step 2000: 0.762999
Minibatch accuracy: 84.4%
Validation accuracy: 87.8%
Minibatch loss at step 2500: 0.482414
Minibatch accuracy: 92.2%
Validation accuracy: 88.1%
Minibatch loss at step 3000: 0.632394
Minibatch accuracy: 88.3%
Validation accuracy: 88.7%
Test accuracy: 94.3%


The initialization of weights is very crucial, but what's the influence of different stddev? When the stddev is not properly set, the loss might proceed to inf or nan.. Why?